In [20]:
import yfinance as yahooFinance

GetFacebookInformation = yahooFinance.Ticker("AAPL")
print(GetFacebookInformation.history(period="10y"))

ERROR:yfinance:Failed to get ticker 'AAPL' reason: 'str' object has no attribute 'get'
ERROR:yfinance:$AAPL: possibly delisted; No price data found  (period=10y)


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


In [23]:
import numpy as np

# --- SEQUENCE A (Paris)

day_1 = [10, 25, 60]  # OBSERVATION 1 [temperature, speed, pollution]
day_2 = [13, 10, 80]  # OBSERVATION 2 [temperature, speed, pollution]
day_3 = [ 9,  5, 90]  # OBSERVATION 3 [temperature, speed, pollution]
day_4 = [ 7,  0, 100]  # OBSERVATION 4 [temperature, speed, pollution]

sequence_a = [day_1, day_2, day_3, day_4]

y_a = [110] # Pollution at day 5

# --- SEQUENCE B (Berlin)
sequence_b = [[25, 20, 30], [26, 24, 50], [28, 20, 80], [22, 3, 110]]
y_b = [125]

# --- SEQUENCE C (London)
sequence_c = [[15, 10, 60], [25, 20, 65], [35, 10, 75], [36, 15, 70]]
y_c = [75]

X = np.array([sequence_a, sequence_b, sequence_c]).astype(np.float32)
y = np.expand_dims(np.array([y_a, y_b, y_c]).astype(np.float32), axis=-1)

print(X.shape)
print(y.shape)

(3, 4, 3)
(3, 1, 1)


In [30]:
# 0- Imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

# 1- RNN Architecture
model = Sequential()
model.add(layers.SimpleRNN(units=2, activation='tanh', input_shape=(4,3)))
model.add(layers.Dense(1, activation="linear"))

# 2- Compilation
model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.1)) # very high lr so we can converge with such a small dataset

# 3- Fit
model.fit(X, y, epochs=1000, verbose=0)

# 4- Predict
model.predict(X) # One prediction per city

/home/joe/.pyenv/versions/3.10.6/envs/lwhf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step


array([[103.154526],
       [103.154526],
       [103.154526]], dtype=float32)